In [1]:
%load_ext autoreload 
%autoreload 2

In [29]:
from gpt3forchem.api_wrappers import query_gpt3, fine_tune, extract_regression_prediction, extract_prediction
from gpt3forchem.data import get_mof_data, discretize
from gpt3forchem.input import create_single_property_forward_prompts_multiple_targets, create_single_property_forward_prompts_regression, create_single_property_forward_prompts
from gpt3forchem.baselines import XGBClassificationBaseline

import os 
from pathlib import Path 
import time 
from sklearn.model_selection import train_test_split
from pycm import ConfusionMatrix

import matplotlib.pyplot as plt
plt.style.use(['science', 'nature'])



In [4]:
data = get_mof_data()

/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/gpt3forchem/data.py:73: DtypeWarning: Columns (23,24,25,26,27,35,36,37,38,39,47,48,49,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  return HashableDataFrame(pd.read_csv(os.path.join(datadir, "mof.csv")))


In [5]:
data.head()

,flavor.all,flavor.gcmc,flavor.csd,flavor.csd-gcmc,outputs.pbe.energy_total,outputs.pbe.energy_vdw,outputs.pbe.energy_elec,outputs.pbe.net_magmom,outputs.pbe.bandgap,outputs.pbe.cbm,...,color,habit,mp,name,formula,density,r_factor,title,powder_study,basename
0,True,False,False,False,-3372.0,-35.84,-3336.0,2.0,0.2255,0.17100,...,green,block,NaN,DULJOM,(C62 H42 Cu2 N4 O12)n,1.318,7.34,NaN,False,DULJOM
1,True,False,True,False,-3322.0,-27.75,-3294.0,0.0,2.8700,0.07135,...,yellow,block,NaN,DARQOF,"(C51 H44 N8 O18 Zn4)n,3(C3 H7 N1 O1)",1.006,6.64,NaN,False,DARQOF
2,True,False,False,False,-3314.0,-36.75,-3276.0,0.0,3.1910,2.53700,...,colorless,block,NaN,BUQROX,"(C56 H48 N8 O10 Zn2)n,6(H2 O1)",1.469,3.99,NaN,False,BUQROX
3,True,True,False,False,-3298.0,-37.84,-3260.0,0.0,2.0530,2.34400,...,NaN,block,NaN,HAWWIN,"(C30 H30 N8 Nd2 O16 S3)n,3n(C3 H6 N2 O1)",1.819,3.69,NaN,False,HAWWIN
4,True,True,False,False,-3272.0,-33.03,-3238.0,104.0,0.2050,-1.45700,...,orange,Cube,NaN,TOWLOI,(C24 H18 Fe3 O16)n,1.380,16.87,NaN,False,TOWLOI


In [54]:
[f for f in data.columns if f.startswith("outputs")]

['outputs.pbe.energy_total',
 'outputs.pbe.energy_vdw',
 'outputs.pbe.energy_elec',
 'outputs.pbe.net_magmom',
 'outputs.pbe.bandgap',
 'outputs.pbe.cbm',
 'outputs.pbe.vbm',
 'outputs.pbe.directgap',
 'outputs.pbe.bandgap_spins',
 'outputs.pbe.cbm_spins',
 'outputs.pbe.vbm_spins',
 'outputs.pbe.directgap_spins',
 'outputs.hle17.energy_total',
 'outputs.hle17.energy_vdw',
 'outputs.hle17.energy_elec',
 'outputs.hle17.net_magmom',
 'outputs.hle17.bandgap',
 'outputs.hle17.cbm',
 'outputs.hle17.vbm',
 'outputs.hle17.directgap',
 'outputs.hle17.bandgap_spins',
 'outputs.hle17.cbm_spins',
 'outputs.hle17.vbm_spins',
 'outputs.hle17.directgap_spins',
 'outputs.hse06_10hf.energy_total',
 'outputs.hse06_10hf.energy_vdw',
 'outputs.hse06_10hf.energy_elec',
 'outputs.hse06_10hf.net_magmom',
 'outputs.hse06_10hf.bandgap',
 'outputs.hse06_10hf.cbm',
 'outputs.hse06_10hf.vbm',
 'outputs.hse06_10hf.directgap',
 'outputs.hse06_10hf.bandgap_spins',
 'outputs.hse06_10hf.cbm_spins',
 'outputs.hse06_10h

### Try classification

In [6]:
features = [
    f for f in data.columns if 'feature' in f
]

In [12]:
discretize(data, 'outputs.pbe.bandgap')

In [13]:
train, test = train_test_split(data, train_size=1000, random_state=42, stratify=data['outputs.pbe.bandgap_cat'])

In [18]:
train_prompts = create_single_property_forward_prompts(train, 'outputs.pbe.bandgap_cat', {'outputs.pbe.bandgap_cat': 'bandgap'}, prompt_prefix='', representation_col='info.mofid.mofid_clean')

In [20]:
test_prompts = create_single_property_forward_prompts(test, 'outputs.pbe.bandgap_cat', {'outputs.pbe.bandgap_cat': 'bandgap'}, prompt_prefix='', representation_col='info.mofid.mofid_clean')

In [21]:
test_prompts

,prompt,completion,repr
0,what is the bandgap of C(Cn1cncc1)CCn1cncc1.[C...,2@@@,C(Cn1cncc1)CCn1cncc1.[Cd].sql.cat0
1,what is the bandgap of [O-]C(=O)c1cccc(c1C(=O)...,0@@@,[O-]C(=O)c1cccc(c1C(=O)[O-])N(=O)=O.[OH2][Ni]....
2,what is the bandgap of [Ag][Ag].[O]P(=O)(CC1=C...,1@@@,[Ag][Ag].[O]P(=O)(CC1=CC=C[CH]N1)[O].[O]P(=O)(...
3,what is the bandgap of [O-]C(=O)C1CC(C(=O)[O-]...,3@@@,[O-]C(=O)C1CC(C(=O)[O-])C(CC1C(=O)[O-])C(=O)[O...
4,what is the bandgap of [Mn].[O-]C=O.met.cat0###,0@@@,[Mn].[O-]C=O.met.cat0
...,...,...,...
2622,what is the bandgap of CCCn1cnc2c1[N]C=N[C]2Cl...,1@@@,CCCn1cnc2c1[N]C=N[C]2Cl.[Ag].dia-f.cat0
2623,what is the bandgap of N#C[C](c1c(F)c(F)c(c(c1...,1@@@,N#C[C](c1c(F)c(F)c(c(c1F)F)[C](C#N)C#N)C#N.[Zn...
2624,what is the bandgap of [Ag].[O-]C(=O)c1cnccn1....,1@@@,[Ag].[O-]C(=O)c1cnccn1.cdt.cat0
2625,what is the bandgap of Cc1cccnc1C1=NN=C([N]1)c...,2@@@,Cc1cccnc1C1=NN=C([N]1)c1ccncc1.[Zn].nbo.cat0


In [23]:
train_size = len(train_prompts)
test_size = len(test_prompts)

filename_base = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
train_filename = (
    f"run_files/{filename_base}_train_prompts_mof_pbe_{train_size}.jsonl"
)
valid_filename = (
    f"run_files/{filename_base}_valid_prompts_mof_pbe_{test_size}.jsonl"
)

train_prompts.to_json(train_filename, orient="records", lines=True)
test_prompts.to_json(valid_filename, orient="records", lines=True)

In [24]:
fine_tune(train_filename, valid_filename, 'ada')

Fine-tune ft-ClmztxlYGsVE3rImEq9XQ6fp has the status "pending" and will not be logged
🎉 wandb sync completed successfully


'ada:ft-lsmoepfl-2022-09-09-15-25-56'

In [25]:
completions = query_gpt3('ada:ft-lsmoepfl-2022-09-09-15-25-56', test_prompts)

In [30]:
predictions = [int(extract_prediction(completions, i)) for i in range(len(completions['choices']))]

In [32]:
test_prompts

,prompt,completion,repr
0,what is the bandgap of C(Cn1cncc1)CCn1cncc1.[C...,2@@@,C(Cn1cncc1)CCn1cncc1.[Cd].sql.cat0
1,what is the bandgap of [O-]C(=O)c1cccc(c1C(=O)...,0@@@,[O-]C(=O)c1cccc(c1C(=O)[O-])N(=O)=O.[OH2][Ni]....
2,what is the bandgap of [Ag][Ag].[O]P(=O)(CC1=C...,1@@@,[Ag][Ag].[O]P(=O)(CC1=CC=C[CH]N1)[O].[O]P(=O)(...
3,what is the bandgap of [O-]C(=O)C1CC(C(=O)[O-]...,3@@@,[O-]C(=O)C1CC(C(=O)[O-])C(CC1C(=O)[O-])C(=O)[O...
4,what is the bandgap of [Mn].[O-]C=O.met.cat0###,0@@@,[Mn].[O-]C=O.met.cat0
...,...,...,...
2622,what is the bandgap of CCCn1cnc2c1[N]C=N[C]2Cl...,1@@@,CCCn1cnc2c1[N]C=N[C]2Cl.[Ag].dia-f.cat0
2623,what is the bandgap of N#C[C](c1c(F)c(F)c(c(c1...,1@@@,N#C[C](c1c(F)c(F)c(c(c1F)F)[C](C#N)C#N)C#N.[Zn...
2624,what is the bandgap of [Ag].[O-]C(=O)c1cnccn1....,1@@@,[Ag].[O-]C(=O)c1cnccn1.cdt.cat0
2625,what is the bandgap of Cc1cccnc1C1=NN=C([N]1)c...,2@@@,Cc1cccnc1C1=NN=C([N]1)c1ccncc1.[Zn].nbo.cat0


In [34]:
true = [int(x.split('@')[0]) for x in test_prompts['completion']]

In [35]:
cm = ConfusionMatrix(actual_vector=true, predict_vector=predictions)

In [36]:
print(cm)

Predict   0         1         2         3         4         
Actual
0         662       95        24        3         0         

1         106       379       218       13        0         

2         11        155       740       36        0         

3         0         3         90        74        0         

4         0         0         6         12        0         





Overall Statistics : 

95% CI                                                            (0.68871,0.72355)
ACC Macro                                                         0.88245
ARI                                                               0.4013
AUNP                                                              0.78876
AUNU                                                              0.71799
Bangdiwala B                                                      0.54
Bennett S                                                         0.63266
CBA                                                               0.500

In [38]:
mof_features = [f for f in data.columns if 'feature' in f]

In [40]:
baseline = XGBClassificationBaseline(None)
baseline.fit(train[mof_features], train['outputs.pbe.bandgap_cat'])

In [41]:
baseline_predictions = baseline.predict(test[mof_features])

In [48]:
baseline_cm = ConfusionMatrix(actual_vector=test['outputs.pbe.bandgap_cat'].to_numpy(), predict_vector=baseline_predictions)

In [49]:
print(baseline_cm)

Predict          large            medium           small            very large       very small       
Actual
large            46               84               11               1                25               

medium           16               569              161              0                196              

small            13               256              235              2                210              

very large       5                6                0                2                5                

very small       15               266              165              1                337              





Overall Statistics : 

95% CI                                                            (0.43357,0.47164)
ACC Macro                                                         0.78104
ARI                                                               0.05661
AUNP                                                              0.60121
AUNU                                 

#### Henry coefficient

In [ ]:
discretize(data, 'outputs.pbe.bandgap')